#### Import Libraries 

In [1]:
import os 
import xarray as xr 
import matplotlib.pyplot as plt 
import torch

#### Paths

In [2]:
path_glob = os.path.join('..', 'data', 'ERA5', 'preprocessed_separate', 'train')
path_reg = os.path.join('..', 'data', 'CERRA', 'preprocessed_separate', 'train')
path = '/media/server/code4earth/Final_Dataset/3_hourly_data_extended'

#### Open Data

In [3]:
#ds_glob = xr.open_dataset(os.path.join(path_glob, 'train_input_scaled.nc'))
#ds_reg = xr.open_dataset(os.path.join(path_reg, 'train_output_scaled.nc'))

ds_glob = xr.open_dataset(os.path.join(path, 'train_input.nc'))
ds_reg = xr.open_dataset(os.path.join(path, 'train_output.nc'))

In [4]:
ds_reg['pm2p5_conc'].shape

(8768, 420, 700)

In [5]:
ds_glob['pm2p5'].shape

(8768, 106, 176)

#### Calcule full statistics

In [6]:
stats_dict = {
    'mean': [],
    'std': [],
    'min': [],
    'max': []
}

for var_name in ds_glob.data_vars:
    var = ds_glob[var_name]
    stats_dict['mean'].append(var.mean().item())
    stats_dict['std'].append(var.std().item())
    stats_dict['min'].append(var.min().item())
    stats_dict['max'].append(var.max().item())

print(stats_dict)

torch.save(stats_dict, 'stats_glob.pt')

{'mean': [0.7519739866256714, -0.09553877264261246, 279.32879638671875, 284.2333984375, 11.263875961303711, 664.7784423828125], 'std': [3.796070098876953, 3.8166873455047607, 81.98612976074219, 86.61475372314453, 9.774090766906738, 537.815673828125], 'min': [-27.54461669921875, -32.669921875, 226.4409942626953, 232.0336456298828, 0.0, 8.015374183654785], 'max': [31.538818359375, 28.28443145751953, 301.724853515625, 324.31298828125, 3650.934814453125, 6559.9345703125]}


In [7]:
stats_dict = {
    'mean': [],
    'std': [],
    'min': [],
    'max': []
}

for var_name in ds_reg.data_vars:
    var = ds_reg[var_name]
    stats_dict['mean'].append(var.mean().item())
    stats_dict['std'].append(var.std().item())
    stats_dict['min'].append(var.min().item())
    stats_dict['max'].append(var.max().item())

print(stats_dict)

torch.save(stats_dict, 'stats_reg.pt')

{'mean': [6.025162220001221], 'std': [4.795845031738281], 'min': [0.013366781175136566], 'max': [2189.18017578125]}


In [8]:
for i in range(len(stats_dict['max'])):
    print(stats_dict['mean'][i])
    print(stats_dict['std'][i])
    print(stats_dict['min'][i])
    print(stats_dict['max'][i])
    print()

6.025162220001221
4.795845031738281
0.013366781175136566
2189.18017578125



#### Calcule PerPixel statistics

In [9]:
ds_glob

<xarray.Dataset>
Dimensions:     (time: 8768, latitude: 106, longitude: 176)
Coordinates:
    valid_time  (time) datetime64[ns] ...
  * time        (time) datetime64[ns] 2021-04-01 ... 2024-09-30T21:00:00
  * latitude    (latitude) float64 72.0 71.6 71.2 70.8 ... 31.2 30.8 30.4 30.0
  * longitude   (longitude) float64 -25.0 -24.6 -24.2 -23.8 ... 44.2 44.6 45.0
Data variables:
    u10         (time, latitude, longitude) float32 2.487 2.201 ... -7.625
    v10         (time, latitude, longitude) float32 -0.08188 0.2462 ... 1.799
    d2m         (time, latitude, longitude) float32 247.2 248.7 ... 277.4 280.9
    t2m         (time, latitude, longitude) float32 256.8 261.1 ... 303.5 303.6
    pm2p5       (time, latitude, longitude) float32 7.043 7.736 ... 127.0 133.7
    blh         (time, latitude, longitude) float32 73.28 55.47 ... 592.5 546.9
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-16T14:47 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
mean_glob = ds_glob.mean(dim='time')
std_glob = ds_glob.std(dim='time')
min_glob = ds_glob.min(dim='time')
max_glob = ds_glob.max(dim='time')

In [11]:
combined_glob = xr.concat([mean_glob, std_glob, min_glob, max_glob], dim='stat')
combined_glob['stat'] = ['mean', 'std', 'min', 'max']

In [12]:
mean_reg = ds_reg.mean(dim='time')
std_reg = ds_reg.std(dim='time')
min_reg = ds_reg.min(dim='time')
max_reg = ds_reg.max(dim='time')

In [13]:
combined_reg = xr.concat([mean_reg, std_reg, min_reg, max_reg], dim='stat')
combined_reg['stat'] = ['mean', 'std', 'min', 'max']

In [14]:
combined_reg.sel(stat='mean').to_array().values.shape

(1, 420, 700)

In [15]:
combined_glob.sel(stat='mean').to_array().values.shape

(6, 106, 176)

In [16]:
combined_glob.to_netcdf('stats_pp_glob.nc')

In [17]:
combined_reg.to_netcdf('stats_pp_reg.nc')